# Optimization Tutorial

### Disclaimer
Currently symforce supports generating arbitrary functions from symbolic expressions, and has a C++ optimizer for these functions, but no optimizer that's usable from a notebook.  So the below example is **not** how you should do optimization with SymForce; instead, take a look at `symforce/examples/bundle_adjustment`, until Aaron updates this tutorial with Cling or a Python optimizer.

-------------------------------

Anyway, the idea in this tutorial is that users can generate expressions needed to do optimization (e.g. jacobians, hessions, etc.), and then use the resulting functions with existing optimization software. In this example we generate a function that computes an update to an optimization variable using gradient descent.

Here we demonstrate a simple gradient descent example to show how the basic idea works. In this problem we will use on-manifold gradient descent to minimize the error between two rotations R0 and R1. We assume R0 is constant, and that an initial guess for R1 is given.

In [ ]:
import os
import numpy as np

from symforce import codegen
from symforce import geo
from symforce import sympy as sm
from symforce.values import Values

from symforce.notebook_util import display_code_file

In [ ]:
# Create symbolic rotations (values will be filled at runtime)
R0 = geo.Rot3.symbolic("R0")
R1 = geo.Rot3.symbolic("R1")
epsilon = sm.Symbol("epsilon")  # Small number to prevent numerical errors
alpha = sm.Symbol("alpha")  # Gradient descent step size

In [ ]:
# Compute the error between the two rotations in the tangent space of R1
error = geo.Matrix(R1.local_coordinates(R0, epsilon))  # Vector in tangent space

# To match the traditional gradient descent formulation, we use a scalar error term + gradient
scalar_error = geo.Matrix([error.squared_norm()])
gradient = scalar_error.jacobian(R1)  # Compute the gradient wrt the tangent space of R1

# Here we compute the update to R1 by performing a gradient descent step in the tangent space of R1
current_state = geo.Matrix(R1.local_coordinates(R0, epsilon))
updated_state = current_state - alpha * gradient.T
updated_R1 = R1.retract(updated_state, epsilon)

In [ ]:
# Generate the update function

# Set up inputs and outputs
inputs = Values()
inputs["R0"] = R0
inputs["R1"] = R1
inputs["epsilon"] = epsilon
inputs["alpha"] = alpha

outputs = Values()
outputs["R1_out"] = updated_R1

namespace = "update_function"
# Create the output function
update_function = codegen.Codegen(
    name="update_function",
    inputs=inputs,
    outputs=outputs,
    config=codegen.PythonConfig(),
    return_key="R1_out",
)
# Output the code
update_function_data = update_function.generate_function(namespace=namespace)
display_code_file(update_function_data.function_dir / "update_function.py", "python")

Next we set up the problem and solve

In [ ]:
from sym import Rot3

# Import the generated function
gen_module = codegen.codegen_util.load_generated_package(
    namespace, update_function_data.function_dir
)

R0 = Rot3.from_tangent([-1.7, 0.5, 0.3])
R1 = Rot3.from_tangent([1.5, 0.2, -0.4])
epsilon = 1e-9
alpha = 0.1

print(f"Desired rotation: {R0}")
print(f"Initial rotation: {R1}")
print(f"Initial error: {np.linalg.norm(R1.local_coordinates(R0))}")

In [ ]:
# Run 10 steps of gradient descent
for i in range(10):
    R1 = gen_module.update_function(R0, R1, epsilon, alpha)
    print(f"New error: {np.linalg.norm(R1.local_coordinates(R0))}")
print(f"Optimized rotation: {R1}")